In [132]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

In [133]:
# Lendo as planilhas e também unindo as sheets
sheet_df = pd.read_excel('/home/pves/Documentos/Workspace/heart_attack_analysis/data/Dados_para_TN_vpd.xlsx', sheet_name=None)
dataset_list = []

for key in sheet_df.keys():
    df = sheet_df[key]
    # Dropando as linhas do dataframe que são Nan
    df.dropna(inplace=True, axis=0)
    print(df.shape)
    dataset_list.append(df)
# Concatena todos os dataframes em um único lateralmente
df_vpd = pd.concat(dataset_list, axis=1)
print(df_vpd.shape)

(171, 4)
(171, 4)
(171, 4)
(171, 4)
(171, 4)
(171, 4)
(171, 4)
(171, 4)
(171, 4)
(171, 2)
(171, 38)


In [134]:
# Verificando a estrutura dos dataframes e o tipo de dado que cada um possui
df_vpd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 0 to 170
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Potencial (V)  171 non-null    float64
 1   D              171 non-null    float64
 2   E              171 non-null    float64
 3   F              171 non-null    float64
 4   Potencial (V)  171 non-null    float64
 5   J              171 non-null    float64
 6   K              171 non-null    float64
 7   L              171 non-null    float64
 8   Potencial (V)  171 non-null    float64
 9   M              171 non-null    float64
 10  N              171 non-null    float64
 11  O              171 non-null    float64
 12  Potencial (V)  171 non-null    float64
 13  P              171 non-null    float64
 14  Q              171 non-null    float64
 15  R              171 non-null    float64
 16  Potencial (V)  171 non-null    float64
 17  S              171 non-null    float64
 18  T              

In [135]:
concentrations_1 = {
    'D': 20e-9,
    'E': 20e-9,
    'F': 20e-9,
    'J': 10e-9,
    'K': 10e-9,
    'L': 10e-9,
    'M': 4e-9,
    'N': 4e-9,
    'O': 4e-9,
    'P': 2e-9,
    'Q': 2e-9,
    'R': 2e-9,
    'S': 1e-9,
    'T': 1e-9,
    'U': 1e-9,
    'A': 0.25e-9,
    'B': 0.25e-9,
    'C': 0.25e-9,
    'G': 0.00001e-9,
    'H': 0.00001e-9,
    'I': 0.00001e-9,
    '2,88ng/mL': 2.88e-9
}

In [148]:
target = []
values = []

for col in df_vpd.columns:
    if col in concentrations_1.keys():
        target.append(concentrations_1[col])
        current_value = df_vpd.loc[1:,col].values.reshape(1,170)
        values.append(current_value)

targets = torch.from_numpy(np.array(target, dtype=np.double).reshape(-1,1))
values_torch = torch.from_numpy(np.array(values))

print(targets.shape)
print(values_torch.shape)


torch.Size([22, 1])
torch.Size([22, 1, 170])


In [149]:
values_torch = values_torch[::3]
targets = targets[::3]

In [150]:
for tensor in range(values_torch.shape[0]):
    #values_torch[tensor, 0,:] = (values_torch[tensor, 0,:] - values_torch[tensor, 0,:].min()) / (torch.max(values_torch[tensor, 0,:]) - values_torch[tensor, 0,:].min())
    values_torch[tensor, 0,:] = values_torch[tensor, 0,:]  / torch.max(values_torch[tensor, 0,:])

#targets = (targets - targets.min()) / (targets.max() - targets.min())
targets = targets / targets.max()

print(targets.max())
print(values_torch.max())

tensor(1., dtype=torch.float64)
tensor(1., dtype=torch.float64)


In [151]:
# Separando em treino e teste
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(values_torch, targets, random_state=0, test_size=0.2)

print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: torch.Size([6, 1, 170])
y_train shape: torch.Size([6, 1])
x_test shape: torch.Size([2, 1, 170])
y_test shape: torch.Size([2, 1])


In [152]:
class Conv1DModel(nn.Module):
    def __init__(self, input_size, num_classes, input_leght):
        super(Conv1DModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size = 3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size = 2)
        self.fc1 = nn.Linear(128 * 41, 256)  # Adjust the input size based on your data
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.relu(self.conv1(x))        # in.shape = (1,170) out.shape = (64,168)
        x = self.pool(x)                    # in.shape = (64, 168) out.shape = (64, 84)
        x = self.relu(self.conv2(x))        # in.shape = (64,84) out.shape = (128,82)
        x = self.pool(x)                    # in.shape = (128,82) out.shape = (128,41)
        x = x.view(x.size(0), -1)           # Flatten the tensor                          
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [158]:
input_size = 1          # Assuming a 1D signal
num_classes = 1         # For regression, num_classes should be 1

model = Conv1DModel(input_size, num_classes, values_torch.shape[2])
criterion = nn.MSELoss()                                    # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)        # You can choose an appropriate optimizer and learning rate

In [159]:
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(values_torch.float())
    loss = criterion(output, targets.float())
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/100], Loss: 0.18290606141090393
Epoch [2/100], Loss: 0.11374883353710175
Epoch [3/100], Loss: 0.1030985489487648
Epoch [4/100], Loss: 0.10860991477966309
Epoch [5/100], Loss: 0.10351710021495819
Epoch [6/100], Loss: 0.10632438957691193
Epoch [7/100], Loss: 0.10334672778844833
Epoch [8/100], Loss: 0.10524137318134308
Epoch [9/100], Loss: 0.10283786803483963
Epoch [10/100], Loss: 0.10463567823171616
Epoch [11/100], Loss: 0.10308453440666199
Epoch [12/100], Loss: 0.10330342501401901
Epoch [13/100], Loss: 0.10355386137962341
Epoch [14/100], Loss: 0.10251829028129578
Epoch [15/100], Loss: 0.1030983254313469
Epoch [16/100], Loss: 0.10265407711267471
Epoch [17/100], Loss: 0.10236351937055588
Epoch [18/100], Loss: 0.10271915793418884
Epoch [19/100], Loss: 0.102147676050663
Epoch [20/100], Loss: 0.10217954963445663
Epoch [21/100], Loss: 0.102179616689682
Epoch [22/100], Loss: 0.101688452064991
Epoch [23/100], Loss: 0.10188589990139008
Epoch [24/100], Loss: 0.1016160324215889
Epoch [25/

In [161]:
from metrics_regression import Metrics_Regression
my_metrics = Metrics_Regression(model, values_torch, targets)

print(my_metrics.calc_mean_absolute_error())
print(my_metrics.calc_mean_squared_error())
print(my_metrics.calc_root_mean_squared_error())
print(my_metrics.calc_r_squared())

0.18015104017859698
0.059303203601122886
0.24352249095540002
0.42152860805701553
